In [ ]:
import cobra
from cobra.io import read_sbml_model
from pprint import pprint
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from cobra.medium import minimal_medium

from process_bigraph import Composite
from process_bigraph import ProcessTypes
from process_bigraph.emitter import gather_emitter_results

from cdFBA import register_types
from cdFBA.processes.dfba import dFBA, UpdateEnvironment, StaticConcentration, Injector, WaveFunction

from cdFBA.utils import make_cdfba_composite, get_injector_spec, get_wave_spec, get_static_spec, set_concentration, set_kinetics, get_exchanges

from in_silico_functions.carbon_sources import (load, met_info, find_fluxes, make_fluxes_dict, make_dict,main, classify_met)

In [ ]:
# minimal media (have NO other carbon source besides cholate)

In [ ]:
b_thetaiotaomicron = cobra.io.read_sbml_model(
    "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Bacteroides_thetaiotaomicron_VPI_5482.xml")
b_thetaiotaomicron.optimize()
b_thetaiotaomicron.summary()
e_rectale = cobra.io.read_sbml_model(
    "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Eubacterium_rectale_ATCC_33656.xml")
# e_rectale.optimize()
# e_rectale.summary()
m_simithii = cobra.io.read_sbml_model(
    "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Methanobrevibacter_smithii_ATCC_35061.xml")
# m_simithii.optimize()
# m_simithii.summary()

In [ ]:
b_thetaiotaomicron_summary = b_thetaiotaomicron.summary()
b_thetaiotaomicron_summary_df = b_thetaiotaomicron_summary.to_frame()
print(b_thetaiotaomicron_summary_df)

In [ ]:
# carbon sources for individual bacteria
carbon_sources_b_thetaiotaomicron = classify_met(b_thetaiotaomicron, b_thetaiotaomicron.optimize())
carbon_sources_e_rectale = classify_met(e_rectale, e_rectale.optimize())
carbon_sources_m_simithii = classify_met(m_simithii, m_simithii.optimize())

In [ ]:
def get_exchanges(dict):
    "get all exchanges from a nested dictionary"
    exchanges = []
    for key, value, in dict.items():
        for v in value:
            if "(e)" in v: # (e) exchange reaction naming convention is only for Agora models
                exchanges.append(v)
    return exchanges

In [ ]:
carbon_sources_b_thetaiotaomicron_exchanges =get_exchanges(carbon_sources_b_thetaiotaomicron) # These are all the exchange reactions that this particular bacteria uses. We need to knock these out of solution so that the bacteria does not have access to these as food sources.
carbon_sources_e_rectale_exchanges =get_exchanges(carbon_sources_e_rectale)
carbon_sources_m_simithii_exchanges =get_exchanges(carbon_sources_m_simithii)
# Make "minimal media" my setting reaction bounds to 0 for these exchange reactions

In [ ]:
gut_models = {
        "B_thetaiotaomicron": "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Bacteroides_thetaiotaomicron_VPI_5482.xml",
        "E_rectale": "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Eubacterium_rectale_ATCC_33656.xml",
        "Methanobrevibacter_smithii": "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Methanobrevibacter_smithii_ATCC_35061.xml"
    }

In [ ]:
def make_dict(species, exchanges):
    dict = {}
    dict[species] = exchanges
    return dict

In [ ]:
b_thetaiotaomicron_dict = make_dict("B_thetaiotaomicron", carbon_sources_b_thetaiotaomicron_exchanges)
e_rectale_dict = make_dict("E_rectale", carbon_sources_e_rectale_exchanges)
m_simithii_dict = make_dict("Methanobrevibacter_smithii", carbon_sources_m_simithii_exchanges)

In [ ]:
all_species = {**b_thetaiotaomicron_dict, **e_rectale_dict, **m_simithii_dict}

In [ ]:
exchanges = ['EX_ac(e)', 'EX_but(e)', 'EX_ch4(e)', 'EX_pect(e)'] # acetate, butyrate, CO2, H2, methane, pectins

volume = 2

# dFBA model
spec = make_cdfba_composite(gut_models, medium_type=None, exchanges=exchanges, volume=volume, interval=0.1)

pprint(spec)

In [ ]:
# spec["Species"]['B_thetaiotaomicron']["config"]["changes"]["reaction_knockout"] = []

In [ ]:
# spec["Species"]['E_rectale']["config"]["changes"]["reaction_knockout"] = []

In [ ]:
# spec["Species"]['Methanobrevibacter_smithii']["config"]["changes"]["reaction_knockout"] = []

In [ ]:
#Set reaction bounds (constrain bounds here to make it so that the exchange reactions can't take place)
spec['Species']['B_thetaiotaomicron']['config']['bounds'] = {
            "EX_o2(e)": {"lower": -2, "upper": None},
            "DM_atp_c_": {"lower": 1, "upper": 1}
        }
spec['Species']['E_rectale']['config']['bounds'] = {
            "EX_o2(e)": {"lower": -2, "upper": None},
            "DM_atp_c_": {"lower": 1, "upper": 1}
    # cut off carbon sources
        }
spec['Species']['Methanobrevibacter_smithii']['config']['bounds'] = {
            "EX_o2(e)": {"lower": -2, "upper": None},
            "DM_atp_c_": {"lower": 1, "upper": 1}
    # cut off carbon sources
        }

In [ ]:
#set external substrate concentrations
concentrations = {      # abundance of polysaccharide, limit acetate so that it has to come from keystone species
    'acetate': 0,
    'pectins': 5,
    'butyrate': 0,
    'Methane': 0
}
set_concentration(spec, concentrations)

In [ ]:
#set kinetics
kinetics = {
    'acetate': (0.5, 5),
    'pectins': (0.2, 7), # reduce km, increase Vmax
    'butyrate': (0.5, 5),
    'Methane': (0.5, 5),
}
for species in gut_models.keys():
    set_kinetics(species, spec, kinetics)
pprint(spec)

In [ ]:
#set emitter specs
spec['emitter'] = {
        "_type": "step",
        "address": "local:ram-emitter",
        "config": {
            "emit": {
                "shared_environment": "any",
                "global_time": "any",
            }
        },
        "inputs": {
            "shared_environment": ["Shared Environment"],
            "global_time": ["global_time"]
        }
    }

In [ ]:
#create the core object
core = ProcessTypes()
#register data types
core = register_types(core)
#register all processes and steps
core.register_process('dFBA', dFBA)
core.register_process('UpdateEnvironment', UpdateEnvironment)
core.register_process('StaticConcentration', StaticConcentration)
core.register_process('WaveFunction', WaveFunction)
core.register_process('Injector', Injector)

In [ ]:
#could run this simulation in a loop (first, create spec with make cdFBA composite function, next, RIGHT before changing spec by adding KO rxn, start a loop where I write line to add KO"
# MAKE A DICTIONARY WITH SPECIES AS KEYS AND ASSOCIATED EXCHANGES AS ITEMS
for species, reactions in all_species.items():
    for rxn in reactions:
        spec["Species"][species]["config"]["changes"]["reaction_knockout"] = [rxn]
        sim = Composite({
            "state": spec,
            },
            core=core
        )

        # ALTERNATIVE: dictionary where keys are names of species and values are list of reactions
        # for species, reactions in dictionary.items():
        #   for rxn in reactions:
        #       spec["Species"][species]["config"]["changes"]["reaction_knockout"] = [rxn]

        #run simulation
        sim.run(20)
        #gather results
        results = gather_emitter_results(sim)[('emitter',)]
        #extract time-series data
        timepoints = []
        for timepoint in results:
            time = timepoint.pop('global_time')
            timepoints.append(time)
        env = [timepoint['shared_environment']['concentrations'] for timepoint in results]
        env_combined = {}
        for d in env:
            for key, value in d.items():
                if key not in env_combined:
                    env_combined[key] = []
                env_combined[key].append(value)
        # results
        #plot results for biomass
        fig, ax = plt.subplots(dpi=300)
        for key, value in env_combined.items():
            if key not in ['acetate', 'pectins', 'carbon dioxide', 'butyrate', 'Methane', 'proton']:
                ax.plot(timepoints, env_combined[key], label=key)
                ax.set_yscale('log')
        plt.xlabel('Time (h)')
        plt.ylabel('Biomass (gDW)')
        plt.legend()
        plt.tight_layout()
        plt.show()
        plt.savefig(f"figures/{species}_{rxn}_knockout.png") # output path for images
        #plot substrates
        # fig, ax = plt.subplots(dpi=300)
        # for key, value in env_combined.items():
        #     if key in ['acetate', 'pectins', 'butyrate', 'Methane']:
        #         ax.plot(timepoints, env_combined[key], label=key)
        #         ax.set_yscale('log')
        # plt.xlabel('Time (h)')
        # plt.ylabel('Substrate Concentration (mM)')
        # plt.legend()
        # plt.tight_layout()
        # plt.show()
        # plt.savefig(f"{species}_{rxn}_knockout.png")

    # look up how to save the figure (something like fig.save)
    # now we can go back and compare results

In [ ]:
#create simulation composite
sim = Composite({
        "state": spec,
        },
        core=core
    )

In [ ]:
#run simulation
sim.run(20)

In [ ]:
#gather results
results = gather_emitter_results(sim)[('emitter',)]

In [ ]:
#extract time-series data
timepoints = []
for timepoint in results:
    time = timepoint.pop('global_time')
    timepoints.append(time)
env = [timepoint['shared_environment']['concentrations'] for timepoint in results]
env_combined = {}
for d in env:
    for key, value in d.items():
        if key not in env_combined:
            env_combined[key] = []
        env_combined[key].append(value)

In [ ]:
results

In [ ]:
#plot results for biomass
fig, ax = plt.subplots(dpi=300)
for key, value in env_combined.items():
    if key not in ['acetate', 'pectins', 'butyrate', 'Methane']:
        ax.plot(timepoints, env_combined[key], label=key)
        ax.set_yscale('log')
plt.xlabel('Time (h)')
plt.ylabel('Biomass (gDW)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#plot substrates
fig, ax = plt.subplots(dpi=300)
for key, value in env_combined.items():
    if key in ['acetate', 'pectins', 'butyrate', 'Methane']:
        ax.plot(timepoints, env_combined[key], label=key)
        ax.set_yscale('log')
plt.xlabel('Time (h)')
plt.ylabel('Substrate Concentration (mM)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# TODO
# Look at just ONE species and see if it's consuming polysaccharide and producing acetate
# Try different polysaccharides and see if a different one is consumed more